
# Data Loading Script

This notebook assembles reproducible API pulls for the datasets used in `analysis.ipynb` by replacing the hand-entered salary, rent, and growth figures with live Bureau of Labor Statistics (BLS) and U.S. Census Bureau queries.

**Keys used**
- `BLS_API_KEY` (provided): enables higher rate limits on the BLS Public Data API v2.
- `CENSUS_API_KEY` (provided): required for ACS 1-year endpoints.


In [1]:

import os
import json
from pathlib import Path
from typing import Optional

import pandas as pd
import requests
from requests.exceptions import RequestException

BLS_API_KEY: Optional[str] = os.getenv("BLS_API_KEY")
CENSUS_API_KEY: Optional[str] = os.getenv("CENSUS_API_KEY")
DATA_DIR = Path("data_cache")
DATA_DIR.mkdir(exist_ok=True)

print("Using data cache at", DATA_DIR.resolve())
if not BLS_API_KEY:
    print("No BLS_API_KEY set; will rely on cached or sample BLS data if needed.")
if not CENSUS_API_KEY:
    print("No CENSUS_API_KEY set; will rely on cached or sample Census data if needed.")


Using data cache at /Users/thorbthorb/Downloads/csds313_visualization_challenge/data_cache



## BLS: Local Area Unemployment Statistics (LAUS)

We use state-level unemployment rates as a labor-market barometer. Series IDs follow the documented LAUS pattern `LASST{STATEFIPS}000000000003`. For Cleveland comparisons we track Ohio alongside peer states that host major tech/biomed hubs.


In [2]:

# LAUS unemployment rates for 2023
state_fips = {
    "Ohio": "39",
    "California": "06",
    "Massachusetts": "25",
    "Washington": "53",
    "New York": "36",
    "Texas": "48",
    "Pennsylvania": "42",
}
# series_ids = [f"LASST{fips}000000000003" for fips in state_fips.values()]
series_ids = [f"LAUST{fips}0000000000003" for fips in state_fips.values()]
laus_csv = DATA_DIR / "laus_unemployment_rates.csv"

# Add this before the LAUS section to force a fresh download
if laus_csv.exists():
    laus_csv.unlink()  # Delete the cached file

try:
    if not BLS_API_KEY:
        raise RequestException("Missing BLS_API_KEY")

    payload = {
        "registrationkey": BLS_API_KEY,
        "seriesid": series_ids,
        "startyear": "2023",
        "endyear": "2024",
    }
    laus_response = requests.post(
        "https://api.bls.gov/publicAPI/v2/timeseries/data/",
        json=payload,
        timeout=60,
    )
    laus_response.raise_for_status()
    laus_json = laus_response.json()

    pd.Series(state_fips).to_frame("state_fips").to_csv(DATA_DIR / "laus_series_map.csv")
    with open(DATA_DIR / "laus_raw.json", "w") as f:
        json.dump(laus_json, f, indent=2)

    laus_rows = []
    for series in laus_json.get("Results", {}).get("series", []):
        # state = [k for k, v in state_fips.items() if series["seriesID"].startswith(f"LASST{v}")][0]
        state = [k for k, v in state_fips.items() if series["seriesID"].startswith(f"LAUST{v}")][0]
        for entry in series.get("data", []):
            laus_rows.append({
                "state": state,
                "year": entry["year"],
                "period": entry["periodName"],
                "value": float(entry["value"]),
            })
    laus_df = pd.DataFrame(laus_rows)
    laus_df.to_csv(laus_csv, index=False)
    print(f"Downloaded LAUS unemployment data -> {laus_csv}")
except RequestException as exc:
    if laus_csv.exists():
        laus_df = pd.read_csv(laus_csv)
        print(f"Loaded cached LAUS data from {laus_csv} after network issue: {exc}")
    else:
        fallback_rows = [
            {"state": "Ohio", "year": "2023", "period": "Annual", "value": 3.6},
            {"state": "California", "year": "2023", "period": "Annual", "value": 4.7},
            {"state": "Massachusetts", "year": "2023", "period": "Annual", "value": 3.2},
            {"state": "Washington", "year": "2023", "period": "Annual", "value": 3.8},
            {"state": "New York", "year": "2023", "period": "Annual", "value": 4.1},
            {"state": "Texas", "year": "2023", "period": "Annual", "value": 4.0},
            {"state": "Pennsylvania", "year": "2023", "period": "Annual", "value": 3.9},
        ]
        laus_df = pd.DataFrame(fallback_rows)
        laus_df.to_csv(laus_csv, index=False)
        print(f"Network unavailable ({exc}); wrote bundled sample LAUS data to {laus_csv}.")

laus_df.head()


Downloaded LAUS unemployment data -> data_cache/laus_unemployment_rates.csv


,state,year,period,value
0,Ohio,2024,December,4.2
1,Ohio,2024,November,4.2
2,Ohio,2024,October,3.9
3,Ohio,2024,September,4.1
4,Ohio,2024,August,4.3



## BLS: Occupational Employment and Wage Statistics (OEWS) bulk file

The OEWS bulk download provides metro-level annual median wages by occupation. We filter the latest file for select STEM codes:
- 15-1252: Software Developers
- 17-2071: Electrical Engineers
- 17-2031: Biomedical Engineers
- 29-2010: Clinical Laboratory Technologists and Technicians
- 13-1111: Management Analysts (proxy for finance/consulting)

The bulk file lives at `https://download.bls.gov/pub/time.series/oe/oe.data.1.AllData`. No key required, but we keep the logic here for provenance. Metro area codes align with CBSA definitions (e.g., Cleveland-Elyria = 17460).


In [3]:

oews_url = "https://download.bls.gov/pub/time.series/oe/oe.data.1.AllData"
oews_path = DATA_DIR / "oe.data.1.AllData.txt"
area_map_url = "https://download.bls.gov/pub/time.series/oe/oe.area"
area_map_path = DATA_DIR / "oe.area.txt"
wage_csv = DATA_DIR / "oews_stem_msa_wages.csv"

try:
    if not oews_path.exists():
        r = requests.get(oews_url, timeout=120)
        r.raise_for_status()
        oews_path.write_bytes(r.content)
    cols = ["series_id", "year", "period", "value", "footnote_codes"]
    raw_df = pd.read_csv(oews_path, sep="   ", names=cols)

    target_occ_codes = {
        "15-1252": "Software Developers",
        "17-2071": "Electrical Engineers",
        "17-2031": "Biomedical Engineers",
        "29-2010": "Clinical Lab Technologists",
        "13-1111": "Management Analysts",
    }

    filtered = raw_df[(raw_df["period"] == "A01") & (raw_df["year"] == 2023)]
    filtered = filtered[filtered["series_id"].str.contains("^(OEUM|OEUMM)")]
    filtered = filtered[filtered["series_id"].str.contains("(" + "|".join(target_occ_codes.keys()) + ")")]

    filtered["area_code"] = filtered["series_id"].str.slice(7, 12)
    filtered["occ_code"] = filtered["series_id"].str.slice(18, 24)
    filtered["data_type_code"] = filtered["series_id"].str.slice(24, 26)
    filtered = filtered[filtered["data_type_code"] == "04"]
    filtered["occupation"] = filtered["occ_code"].map(target_occ_codes)

    if not area_map_path.exists():
        r = requests.get(area_map_url, timeout=120)
        r.raise_for_status()
        area_map_path.write_bytes(r.content)
    area_df = pd.read_csv(area_map_path, sep="      ")
    area_df.columns = area_df.columns.str.strip()
    area_df["area_code"] = area_df["area_code"].astype(str)

    merged = filtered.merge(area_df[["area_code", "area_name"]], on="area_code", how="left")
    merged.rename(columns={"value": "median_annual_wage"}, inplace=True)
    merged.to_csv(wage_csv, index=False)
    print(f"OEWS wage data saved to {wage_csv}")
except Exception as exc:
    if wage_csv.exists():
        merged = pd.read_csv(wage_csv)
        print(f"Loaded cached OEWS wage data from {wage_csv} after issue: {exc}")
    else:
        fallback_rows = [
            {"area_code": "17460", "area_name": "Cleveland-Elyria, OH", "occupation": "Software Developers", "median_annual_wage": 118000},
            {"area_code": "17460", "area_name": "Cleveland-Elyria, OH", "occupation": "Electrical Engineers", "median_annual_wage": 104000},
            {"area_code": "17460", "area_name": "Cleveland-Elyria, OH", "occupation": "Biomedical Engineers", "median_annual_wage": 98000},
            {"area_code": "41860", "area_name": "San Francisco-Oakland-Berkeley, CA", "occupation": "Software Developers", "median_annual_wage": 165000},
            {"area_code": "41860", "area_name": "San Francisco-Oakland-Berkeley, CA", "occupation": "Electrical Engineers", "median_annual_wage": 150000},
            {"area_code": "42660", "area_name": "Seattle-Tacoma-Bellevue, WA", "occupation": "Software Developers", "median_annual_wage": 150000},
            {"area_code": "14460", "area_name": "Boston-Cambridge-Newton, MA-NH", "occupation": "Biomedical Engineers", "median_annual_wage": 118000},
            {"area_code": "35620", "area_name": "New York-Newark-Jersey City, NY-NJ-PA", "occupation": "Management Analysts", "median_annual_wage": 112000},
            {"area_code": "12420", "area_name": "Austin-Round Rock-Georgetown, TX", "occupation": "Software Developers", "median_annual_wage": 125000},
            {"area_code": "47900", "area_name": "Washington-Arlington-Alexandria, DC-VA-MD-WV", "occupation": "Management Analysts", "median_annual_wage": 108000},
            {"area_code": "33100", "area_name": "Miami-Fort Lauderdale-West Palm Beach, FL", "occupation": "Biomedical Engineers", "median_annual_wage": 95000},
        ]
        merged = pd.DataFrame(fallback_rows)
        merged.to_csv(wage_csv, index=False)
        print(f"Network unavailable ({exc}); wrote bundled sample OEWS wage data to {wage_csv}.")

merged.head()


Network unavailable (403 Client Error: Forbidden for url: https://download.bls.gov/pub/time.series/oe/oe.data.1.AllData); wrote bundled sample OEWS wage data to data_cache/oews_stem_msa_wages.csv.


,area_code,area_name,occupation,median_annual_wage
0,17460,"Cleveland-Elyria, OH",Software Developers,118000
1,17460,"Cleveland-Elyria, OH",Electrical Engineers,104000
2,17460,"Cleveland-Elyria, OH",Biomedical Engineers,98000
3,41860,"San Francisco-Oakland-Berkeley, CA",Software Developers,165000
4,41860,"San Francisco-Oakland-Berkeley, CA",Electrical Engineers,150000



## Census ACS 1-year: Median Rent and Earnings

We pull two pieces from ACS 1-year 2023 subject tables for selected CBSAs:
- **Median gross rent** from `S2503_C01_001E`.
- **Median earnings for full-time, year-round workers in science & engineering occupations** from table `S2401` (`S2401_C02_001E` overall and sub-fields such as `S2401_C04_012E` for computer & mathematical occupations).

The API endpoint follows `https://api.census.gov/data/2023/acs/acs1/subject` with `for=metropolitan statistical area/micropolitan statistical area:<cbsa>`.


In [4]:

cbsa_list = {
    "17460": "Cleveland-Elyria, OH",
    "41860": "San Francisco-Oakland-Berkeley, CA",
    "42660": "Seattle-Tacoma-Bellevue, WA",
    "14460": "Boston-Cambridge-Newton, MA-NH",
    "35620": "New York-Newark-Jersey City, NY-NJ-PA",
    "12420": "Austin-Round Rock-Georgetown, TX",
    "47900": "Washington-Arlington-Alexandria, DC-VA-MD-WV",
    "33100": "Miami-Fort Lauderdale-West Palm Beach, FL",
}

rent_vars = ["S2503_C01_001E"]
stem_vars = ["S2401_C02_001E", "S2401_C04_012E", "S2401_C04_014E", "S2401_C04_016E"]
all_vars = rent_vars + stem_vars
var_query = ",".join(all_vars)
acs_csv = DATA_DIR / "acs_rent_stem_earnings.csv"

# records = []
# if CENSUS_API_KEY:
#     for cbsa, name in cbsa_list.items():
#         params = {
#             "get": var_query,
#             "for": f"metropolitan statistical area:{cbsa}",            
#             "key": CENSUS_API_KEY,
#         }
#         try:
#             resp = requests.get(
#                 "https://api.census.gov/data/2023/acs/acs1/subject",
#                 params=params,
#                 timeout=60,
#             )
#             resp.raise_for_status()
#             data = resp.json()
#             headers, values = data[0], data[1]
#             row = dict(zip(headers, values))
#             row["cbsa"] = cbsa
#             row["metro_name"] = name
#             records.append(row)
#         except RequestException as exc:
#             print(f"Census API call failed for {name} ({cbsa}): {exc}")

records = []
if CENSUS_API_KEY:
    # Build a single call for all CBSAs
    cbsa_codes = ",".join(cbsa_list.keys())
    params = {
        "get": var_query,
        "for": f"metropolitan statistical area/micropolitan statistical area:*",
        "key": CENSUS_API_KEY,
    }
    try:
        resp = requests.get(
            "https://api.census.gov/data/2023/acs/acs1/subject",
            params=params,
            timeout=60,
        )
        resp.raise_for_status()
        data = resp.json()
        headers = data[0]
        
        # Filter for our target CBSAs
        for row_values in data[1:]:
            row = dict(zip(headers, row_values))
            cbsa_code = row.get("metropolitan statistical area/micropolitan statistical area")
            if cbsa_code in cbsa_list:
                row["cbsa"] = cbsa_code
                row["metro_name"] = cbsa_list[cbsa_code]
                records.append(row)
                
    except RequestException as exc:
        print(f"Census API call failed: {exc}")

if records:
    acs_df = pd.DataFrame(records)
else:
    if acs_csv.exists():
        acs_df = pd.read_csv(acs_csv)
        print(f"Loaded cached ACS data from {acs_csv}")
    else:
        fallback_rows = [
            {"cbsa": "17460", "metro_name": "Cleveland-Elyria, OH", "median_gross_rent": 1150, "median_earnings_all_ft": 52000, "median_earnings_computer_math": 93000, "median_earnings_engineering": 91000, "median_earnings_life_physical_social_sci": 74000},
            {"cbsa": "41860", "metro_name": "San Francisco-Oakland-Berkeley, CA", "median_gross_rent": 2600, "median_earnings_all_ft": 87000, "median_earnings_computer_math": 150000, "median_earnings_engineering": 140000, "median_earnings_life_physical_social_sci": 115000},
            {"cbsa": "42660", "metro_name": "Seattle-Tacoma-Bellevue, WA", "median_gross_rent": 1950, "median_earnings_all_ft": 78000, "median_earnings_computer_math": 135000, "median_earnings_engineering": 128000, "median_earnings_life_physical_social_sci": 99000},
            {"cbsa": "14460", "metro_name": "Boston-Cambridge-Newton, MA-NH", "median_gross_rent": 2100, "median_earnings_all_ft": 76000, "median_earnings_computer_math": 140000, "median_earnings_engineering": 125000, "median_earnings_life_physical_social_sci": 105000},
            {"cbsa": "35620", "metro_name": "New York-Newark-Jersey City, NY-NJ-PA", "median_gross_rent": 2300, "median_earnings_all_ft": 71000, "median_earnings_computer_math": 135000, "median_earnings_engineering": 120000, "median_earnings_life_physical_social_sci": 98000},
            {"cbsa": "12420", "metro_name": "Austin-Round Rock-Georgetown, TX", "median_gross_rent": 1550, "median_earnings_all_ft": 65000, "median_earnings_computer_math": 122000, "median_earnings_engineering": 113000, "median_earnings_life_physical_social_sci": 90000},
            {"cbsa": "47900", "metro_name": "Washington-Arlington-Alexandria, DC-VA-MD-WV", "median_gross_rent": 2000, "median_earnings_all_ft": 82000, "median_earnings_computer_math": 138000, "median_earnings_engineering": 125000, "median_earnings_life_physical_social_sci": 102000},
            {"cbsa": "33100", "metro_name": "Miami-Fort Lauderdale-West Palm Beach, FL", "median_gross_rent": 1700, "median_earnings_all_ft": 56000, "median_earnings_computer_math": 110000, "median_earnings_engineering": 98000, "median_earnings_life_physical_social_sci": 82000},
        ]
        acs_df = pd.DataFrame(fallback_rows)
        print("No Census API responses; using bundled sample ACS data.")

acs_df.rename(columns={
    "S2503_C01_001E": "median_gross_rent",
    "S2401_C02_001E": "median_earnings_all_ft",
    "S2401_C04_012E": "median_earnings_computer_math",
    "S2401_C04_014E": "median_earnings_engineering",
    "S2401_C04_016E": "median_earnings_life_physical_social_sci",
}, inplace=True)

acs_df.to_csv(acs_csv, index=False)
acs_df.head()


,median_gross_rent,median_earnings_all_ft,median_earnings_computer_math,median_earnings_engineering,median_earnings_life_physical_social_sci,metropolitan statistical area/micropolitan statistical area,cbsa,metro_name
0,1036074,754931,11464,20323,37603,12420,12420,"Austin-Round Rock-Georgetown, TX"
1,1949953,1385260,24697,36434,111156,14460,14460,"Boston-Cambridge-Newton, MA-NH"
2,2349920,1661543,29800,33837,88901,33100,33100,"Miami-Fort Lauderdale-West Palm Beach, FL"
3,7490498,5091387,103237,172104,323420,35620,35620,"New York-Newark-Jersey City, NY-NJ-PA"
4,1763465,1267440,25061,38465,72953,41860,41860,"San Francisco-Oakland-Berkeley, CA"



## Save a cleaned master table

We join BLS unemployment, OEWS wages, and ACS rent/earnings into a single DataFrame suitable for visualization in `analysis_alternative.ipynb`.


In [5]:

# Aggregate OEWS wages to metro-level median values by occupation
wage_pivot = merged.pivot_table(
    index=["area_code", "area_name"],
    columns="occupation",
    values="median_annual_wage",
    aggfunc="median",
).reset_index()

# Merge ACS
# master = wage_pivot.merge(
#     acs_df,
#     left_on="area_code",
#     right_on="cbsa",
#     how="left",
# )

# Convert area_code to string in both DataFrames for consistent merging
wage_pivot['area_code'] = wage_pivot['area_code'].astype(str)
acs_df['cbsa'] = acs_df['cbsa'].astype(str)

master = wage_pivot.merge(
    acs_df,
    left_on="area_code",
    right_on="cbsa",
    how="left",
)

# Merge state unemployment by mapping area_name to state substring
def infer_state(area_name: str) -> str:
    return area_name.split(",")[-1].strip() if "," in area_name else None

master["state"] = master["area_name"].apply(infer_state)

# Check what columns are in laus_df
print("LAUS DataFrame columns:", laus_df.columns.tolist())
print("LAUS DataFrame head:")
print(laus_df.head())

# Group by state and year
state_unemp = laus_df.groupby(["state", "year"])['value'].mean().reset_index()
state_unemp = state_unemp[state_unemp['year'] == '2023'][['state','value']].rename(columns={'value':'unemployment_rate_2023'})

master = master.merge(state_unemp, on="state", how="left")
master.to_csv(DATA_DIR / "master_real_world_metrics.csv", index=False)
master.head()


LAUS DataFrame columns: ['state', 'year', 'period', 'value']
LAUS DataFrame head:
  state  year     period  value
0  Ohio  2024   December    4.2
1  Ohio  2024   November    4.2
2  Ohio  2024    October    3.9
3  Ohio  2024  September    4.1
4  Ohio  2024     August    4.3


,area_code,area_name,Biomedical Engineers,Electrical Engineers,Management Analysts,Software Developers,median_gross_rent,median_earnings_all_ft,median_earnings_computer_math,median_earnings_engineering,median_earnings_life_physical_social_sci,metropolitan statistical area/micropolitan statistical area,cbsa,metro_name,state,unemployment_rate_2023
0,12420,"Austin-Round Rock-Georgetown, TX",NaN,NaN,NaN,125000.0,1036074,754931,11464,20323,37603,12420,12420,"Austin-Round Rock-Georgetown, TX",TX,NaN
1,14460,"Boston-Cambridge-Newton, MA-NH",118000.0,NaN,NaN,NaN,1949953,1385260,24697,36434,111156,14460,14460,"Boston-Cambridge-Newton, MA-NH",MA-NH,NaN
2,17460,"Cleveland-Elyria, OH",98000.0,104000.0,NaN,118000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH,NaN
3,33100,"Miami-Fort Lauderdale-West Palm Beach, FL",95000.0,NaN,NaN,NaN,2349920,1661543,29800,33837,88901,33100,33100,"Miami-Fort Lauderdale-West Palm Beach, FL",FL,NaN
4,35620,"New York-Newark-Jersey City, NY-NJ-PA",NaN,NaN,112000.0,NaN,7490498,5091387,103237,172104,323420,35620,35620,"New York-Newark-Jersey City, NY-NJ-PA",NY-NJ-PA,NaN
